In [1]:
import re
import pandas as pd
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook

In [2]:
#root = os.getcwd()
root = r"D:\adham-till-code"
root

'C:\\Users\\Asus\\Desktop\\Project\\ImageRecommender'

In [3]:
def total_images(main_folder):
    """
    This function is used for getting total number of jpg/png files in the main folder
    """
    # Define the pattern to search for .jpg files recursively
    file_pattern = main_folder + '/**/*.[jp][pn]g'
    
    # Use glob to find all matching files
    my_list = glob.glob(file_pattern, recursive=True)
    
    # just return the number of total files
    return len(my_list)

In [23]:
# creating the generator
def reader(main_folder):
    '''
    This function generates through all the images file in the folder
    '''  
    # Counter for unique image names
    image_counter = 0  # starting index
    print(main_folder)

    # Define the pattern to search for .jpg files recursively
    file_pattern = main_folder + '/**/*.[jp][pn]g'  
    
    # Use glob to find all matching files
    image_files = glob.glob(file_pattern, recursive=True)

    # Process the list of found .jpg files
    for path in image_files:
        # steps for the file. open with cv2 --> if okay --> yield the id, image array and filepath
        img = cv2.imread(path)

        # check if image can be loaded correctly
        if img is not None:   
            #create new name
            unique_id = f"{image_counter:07}_"

            image_counter += 1

            # yield tuple, so later we need to use
            yield (unique_id, img, path)

        else:
            # check path or image format
            print(f"Error while loading image: {path}")

In [24]:
def get_colour_hist(image):
    """
    This function is used to calculate color histogram for image
    """
    red_channel = image[:, :, 0]
    green_channel = image[:, :, 1]
    blue_channel = image[:, :, 2]

    # Calculate colour histogram
    red_hist = cv2.calcHist([red_channel], [0], None, [30], [0, 256])
    green_hist = cv2.calcHist([green_channel], [0], None, [30], [0, 256])
    blue_hist = cv2.calcHist([blue_channel], [0], None, [30], [0, 256])

    # Normalize the colour histogram value
    red_hist = cv2.normalize(red_hist, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    green_hist = cv2.normalize(green_hist, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    blue_hist = cv2.normalize(blue_hist, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    return red_hist, green_hist, blue_hist

In [25]:
def main(filepath, pc="windows"):
    # Counter for unique image names
    gen = reader(filepath)

    master_df = pd.DataFrame()
    # LOOP OVER ALL IMAGES

    for index, image, file_path in tqdm_notebook(gen, desc="extracting RGB information from the images",
                                                total = total_images(filepath)):
        # get the color histogram
        rgb_array = get_colour_hist(image)

        # extract and format the color values
        red_channel = rgb_array[0]
        green_channel = rgb_array[1]
        blue_channel = rgb_array[2]

        # merged the list from RGB channel into one list
        merged_list = np.concatenate((red_channel, green_channel, blue_channel), axis=1).flatten().tolist()

        # create a DataFrame, tranpose it, add id and merge to master_df
        df = pd.DataFrame(merged_list)
        df = df.transpose()
        df["image_id"] = index
        if pc == "windows":
            df["file_path"] = file_path.lstrip(file_path[0:2]) # save the path that are consistent for both macbook and windows
        else:
            df["file_path"] = file_path
        df = df.set_index('image_id')

        master_df = pd.concat([master_df, df])

    # split the path from the color historgram
    path_df = master_df[["file_path"]]
    color_df = master_df.drop("file_path", axis="columns")
    
    # change the path back to project root again
    os.chdir(root)
    
    #save as pickle
    path_df.to_pickle('file_path.pkl')
    color_df.to_pickle('color_histogram.pkl')
    
    print("-------------------------")
    print("Notes:")
    if pc == "windows":
        print("path and colour information saved")
    else:
        print("This path might only works on your pc system. Please consider the changes of path name if you are using another device")

In [31]:
#filepath = os.path.join(os.getcwd(), "weather_image_recognition")
filepath = r"D:\images" # path for image in hard drive

In [32]:
if __name__ == "__main__":
    main(filepath, pc="windows")

extracting RGB information from the images:   0%|          | 0/31943 [00:00<?, ?it/s]

C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition
Error while loading image: C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\fogsmog\4514.jpg
Error while loading image: C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\my_folder\subfolder_1\subfolder_2\4514.jpg
Error while loading image: C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\snow\1187.jpg
-------------------------
Notes:
path and colour information saved


In [33]:
path_df = pd.read_pickle('file_path.pkl')
path_df.head()

,file_path
image_id,
0000000_,\Users\Asus\Desktop\Project\ImageRecommender\w...
0000001_,\Users\Asus\Desktop\Project\ImageRecommender\w...
0000002_,\Users\Asus\Desktop\Project\ImageRecommender\w...
0000003_,\Users\Asus\Desktop\Project\ImageRecommender\w...
0000004_,\Users\Asus\Desktop\Project\ImageRecommender\w...


In [34]:
color_df = pd.read_pickle('color_histogram.pkl')
color_df.shape

(31940, 90)